# Gerador de notícias
1. Identificar URL do Google News
2. Identificar notícias primárias (que servem como fonte para notícias do mesmo assunto)
3. Selecionar a notícia primária como referência
4. Identificar notícias secundárias, com respectivas URLs das notícias
5. Buscar dados de cada uma das notícias (título, texto, imagem, texto da imagem)
6. Identificar parâmetros para gerar a notícia do Jornal Sócrates
7. Criar a notícia do Jornal Sócrates com título, preâmbulo e texto
8. Selecionar imagem para a notícia

In [7]:
import requests
import openai
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pyperclip

%run "./news_source/news.ipynb" import News


load_dotenv()
API_URL = os.getenv("API_URL")
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
class NewsSource:

    def __init__(self, sectionId):
        self.sectionId = sectionId
        self.newsSource, self.sectionName, self.sectionArr, self.tags = self.get_source_news()
        
        # Dados das notícias principais
        self.html_soup = self.get_html(self.newsSource)
        self.main_news = self.get_main_news_list()
        self.selected_news = ''
        
        # Dados das notícias derivadas da principal selecionada
        self.all_news = []
        self.all_images = []
        
        # Dados da notícia a ser gerada
        self.selected_references = []
        self.news_header = ''
        self.news_text = ''
        self.news_image_url = ''
        self.news_image_text = ''
        
        self.output_primary_news_area = widgets.Output()
        self.output_news_len_area = widgets.Output()
        self.get_primary_news()
        # self.image_display = widgets.Output()
    
    
    # Identificar URL do Google News
    def get_source_news(self):
        url = f'{API_URL}/api/newssource'
        newssource = requests.get(url).json()
        gn = [doc for doc in newssource if doc['__t'] == 'google news - html']
        for doc in gn:
            if doc['sectionId'] == self.sectionId:
                return doc['newsSource'], doc['sectionName'], doc['sectionArr'], doc['tags']
        return None, None, None, None


    def get_html(self, url):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        html_content = requests.get(url, headers=browsers)
        soup = BeautifulSoup(html_content.text, 'html.parser')
        return soup

    
    # Identificar notícias primárias (que servem como fonte para notícias do mesmo assunto)
    def get_main_news_list(self):
        news_blocks = self.html_soup.find_all(class_='PO9Zff Ccj79 kUVvS')
        result = []
        for nb in news_blocks:
            try:
                title = nb.find('a', class_='gPFEn').get_text()
                all_news = 'https://news.google.com' + nb.find('div', class_='Ylktk').find('a').get('href')[1:]
                result.append({'title': title, 'url': all_news})
            except:
                pass
        return result


    # Selecioar a notícia primária como referência
    def get_primary_news(self):
        options = [news['title'] for news in self.main_news]
        self.news_checkboxes = widgets.Dropdown(
            options=options,
            description=f'{self.sectionName}:',
            disabled=False,
            layout=widgets.Layout(width='70%')
        )
        self.confirm_primary_news_button = widgets.Button(
            description='Selecionar notícia primária',
            button_style='success',
            icon='check',
            layout=widgets.Layout(width='20%')
        )
        self.confirm_primary_news_button.on_click(self.on_get_primary_news)
        display(self.news_checkboxes, self.confirm_primary_news_button, self.output_primary_news_area)

    def on_get_primary_news(self, b):
        with self.output_primary_news_area:
            clear_output()
            self.selected_news = self.news_checkboxes.value
            print(f"Notícia primária: {self.selected_news}")
            self.get_reference_news()

    def get_reference_news(self):
        all_news_mainUrl = [news for news in self.main_news if news['title'] == self.selected_news][0]['url']
        print(f'Buscando notícias em {all_news_mainUrl}')
        
        # Identificar notícias secundárias, com respectivas URLs das notícias
        reference_news = []
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        try:
            driver.get(all_news_mainUrl)
            WebDriverWait(driver, 15)
            articles = driver.find_elements(By.CSS_SELECTOR, "article")
            for counter, a in enumerate(articles[:10], start=1):
                title = a.find_element(By.CSS_SELECTOR, "h4").text
                share_btn = a.find_element(By.CSS_SELECTOR, '[data-tooltip="Compartilhar"]')
                share_btn.click()
                copy_link_btn = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[aria-label="Copiar link"]')))
                copy_link_btn.click()
                close_dialog_btn = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[aria-label="Fechar caixa de diálogo"]')))
                close_dialog_btn.click()
                url = pyperclip.paste()
                # print(f'#{counter} >> {url} >> {title}')
                reference_news.append({'title': title, 'url': url})
        except Exception as e:
            print("Problema na coleta dos dados da página:", e)
        driver.quit()

        # Buscar dados de cada uma das notícias (título, texto, url da imagem, texto da imagem)
        all_news = []
        for r in reference_news:
            news_data = News(r['url'])
            all_news.append(news_data)
        
        self.selected_references = [news for news in all_news if news.title and news.text]
        self.all_images = [news for news in all_news if news.imageUrl or news.imageText]
        
        self.get_news_params()
    
    
    # Criar a notícia do Jornal Sócrates com título, preâmbulo e texto
    def get_news_params(self):
        # Inputs para os parâmetros
        with self.output_news_len_area:
            clear_output()
            print('Criando notícia para o Jornal Sócrates\n')
            counter = 1
            sum_chars = 0
            sum_words = 0
            for n in self.selected_references:
                n_char = len(n.text)
                n_words = len(n.text.split(' '))
                print(f'#{counter} >> {n.source} >> {n_char} caracteres >> {n_words} palavras >> {n.title}')
                sum_chars += n_char
                sum_words += n_words
                counter += 1
            print(f'### >> {sum_chars} caracteres >> {sum_words} palavras\n\n')
            self.news_title_len = widgets.ToggleButtons(
                options=['entre 75 e 85 caracteres', 'entre 80 e 90 caracteres'],
                description='Título:',
                disabled=False,
                button_style='',
            )
            print(1)
            self.news_summary_len = widgets.ToggleButtons(
                options=['entre 100 e 150 caracteres', 'entre 150 e 200 caracteres', 'entre 200 e 250 caracteres', 'entre 250 e 300 caracteres'],
                description='Preâmbulo:',
                disabled=False,
                button_style='',
            )
            print(2)
            self.news_text_len = widgets.ToggleButtons(
                options=['entre 350 e 400 caracteres', 'entre 1000 e 1200 caracteres', 'entre 1400 e 1600 caracteres', 'entre 2100 e 2200 caracteres'],
                description='Texto:',
                disabled=False,
                button_style='',
            )
            print(3)
            self.create_news_button = widgets.Button(
                description='Criar notícia para o Jornal Sócrates',
                button_style='success',
                icon='check',
                layout=widgets.Layout(width='20%')
            )
            print(4)
            self.create_news_button.on_click(self.on_create_news)
            print(5)
            display(self.news_title_len, self.news_summary_len, self.news_text_len, self.create_news_button, self.output_news_len_area)        
            print(6)
        
    def on_create_news(self, b):
        with self.output_news_len_area:
            print('Criando notícia para o Jornal Sócrates\n')
            print(f'self.news_title_len: {self.news_title_len.value}')
            print(f'self.news_summary_len: {self.news_summary_len.value}')
            print(f'self.news_text_len: {self.news_text_len.value}')
            # self.get_reference_news()
        
        
        # self.create_news_button = widgets.Button(
        #     description='Gerar notícia para o Jornal Socrates',
        #     button_style='success',
        #     tooltip='Gerar notícia para o Jornal Socrates',
        #     icon='check',
        #     layout=widgets.Layout(width='20%')
        # )
        # # self.create_news_button.on_click(self.create_news)
        # display(self.create_news_button, self.output_area)


    def create_news(self, b):
        print('Criando notícia para o Jornal Socrates\n\n')
        
        references = '\n\n'
        for n in self.selected_references:
            references += f'{n.title}\n{n.text}\n\n'
        
        n_chars_header = 400
        n_chars_preamble = 400
        n_chars_text = 400
        
        content = f'Escreva um texto jornalístico que será publicado na seção {self.sectionName} de um jornal de grande circulação.\n'
        content += f'Considere como referêcia explusivamente o que foi extraído de alguns sites de prestígio {references}'
        content += f'Crie o texto sem enrolação e apenas com informações relevantes. Escreva estritamente a notícia a ser publicada.\n\n'
        content += f'O primeiro parágrafo será o título da notícia e deve conter {self.news_title_len}.\n'
        content += f'O segundo parágrafo será o preâmbulo, um resumo da notícia que estimula o leitor a continuar a leitura, e deve conter {self.news_summary_len} caracteres.\n'
        content += f'Os demais parágrafos serão o corpo da notícia e deve conter {self.news_text_len}.\n'
        
        message = [{
            'role': 'user',
            'content': content
        }]
        print(content)
        # response = client.chat.completions.create(
        #     messages=message, 
        #     model="gpt-4o-mini", 
        #     max_tokens=10000, 
        #     temperature=0, 
        # )
        # prediction = response.choices[0].message.content
        # newsData = splitIntoContent(prediction)
        # save_sport(teamName, newsData['title'], newsData['content'])
        # print(f'{teamName} >> {newsData['title']}')


        
    #     with self.output_area:
    #         clear_output()
    #         self.selected_news = self.news_checkboxes.value
    #         print(f"Criando notícia baseado em: {self.selected_news}")
    #         self.create_image_selector()
            
            
            
            
            


    # def create_image_selector(self):
    #     options = [(image.imageUrl, image.imageText) for image in self.all_images]
    #     self.dropdown = widgets.Dropdown(
    #         options=[image_url for image_url, _ in options],
    #         description='Selecione a imagem:',
    #         disabled=False,
    #     )
    #     def on_image_selected(change):
    #         with self.image_display:
    #             self.image_display.clear_output()
    #             selected_url = change['new']
    #             selected_text = next(text for url, text in options if url == selected_url)
    #             display(Image(url=selected_url))
    #             display(widgets.HTML(f"<p>{selected_text}</p>"))
    #             self.news_image_url = selected_url
    #             self.news_image_text = selected_text
    #     self.dropdown.observe(on_image_selected, names='value')
    #     display(self.dropdown, self.image_display)
    #     if options:
    #         on_image_selected({'new': options[0][0]})
        

